Import libraries

In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score
from sklearn.model_selection import GridSearchCV

In [8]:
dataset = pd.read_csv("/content/water_potability.csv");
dataset.head()

NameError: name 'pd' is not defined

Dataset pre-process

Dataset overall information

In [ ]:
dataset.info()

Check precentage of null values

In [ ]:
print(round(dataset.isnull().sum() * 100/dataset.shape[0]))

Set mean value of each column to null values

In [ ]:
dataset['ph'] = dataset['ph'].fillna(dataset['ph'].mean())
dataset['Sulfate'] = dataset['Sulfate'].fillna(dataset['Sulfate'].mean())
dataset['Trihalomethanes'] = dataset['Trihalomethanes'].fillna(dataset['Trihalomethanes'].mean())

In [ ]:
dataset.info()